Draw Statistical Chart

In [1]:
import matplotlib.pyplot as plt
import numpy as np

# Data definition
plt.rc('text', usetex=False)
group = [r'Low $\mathbf{\kappa(c^\top c)}$ and R $>$ 0.8', r'High $\mathbf{\kappa(c^\top c)}$ to Low $\mathbf{\kappa(c^\top c)}$ and R $>$ 0.8', 
        r'High $\mathbf{\kappa(c^\top c)}$ and R $<$ 0.4', "Others"]


categories = ["H1", "WTC11", "H9", "H1-DE"]

data = {
    group[0]: [50.0, 48.2, 46.90, 51.1],
    group[1]: [9.3, 10.3, 10.4, 9.2],
    group[2]: [28.3, 28.6, 29.10, 26.2],
    group[3]: [12.4, 12.9, 13.6, 13.5]
}

# Set bar chart width and colors
bar_width = 0.15
colors = ["#E4CD87", "#EFBC91", "#C69287", "#979998"]

# Create figure and axes
fig, ax = plt.subplots(figsize=(14, 8), dpi = 700)

# Define x-axis positions
index = np.arange(len(categories))

# Plot data for each group, arranged sequentially on x-axis
for i, (group, values) in enumerate(data.items()):
    
    bars = ax.bar(index + i * bar_width, values, bar_width, label=group, color=colors[i])
    
    # Display values above each bar
    for bar, value in zip(bars, values):
        ax.text(
            bar.get_x() + bar.get_width() / 2,  # x position
            bar.get_height() + 1,               # y position (slightly above bar)
            f'{value:.1f}',                     # format label as one decimal
            ha='center', va='bottom',           # center alignment
            fontsize=15, color='black'           # adjust fontsize and color if needed
        )
plt.rcParams['text.usetex'] = False
# Set title and axis labels
#ax.set_xlabel("Four organcells")
ax.set_ylabel(r"Percentage (\%)", fontsize=25)
#ax.set_title("Category Distribution Across Groups")

# Set y-axis range
ax.set_ylim(0, 60)

# Set x-axis ticks and labels, display labels horizontally
ax.set_xticks(index + bar_width * 1.5)
ax.set_xticklabels(categories, rotation=0, ha="center")  # Changed rotation to 0

# Adjust legend position and use Greek letter kappa in title
ax.legend(loc="upper left", bbox_to_anchor=(0.25, 1), borderaxespad=0.5, frameon=False, ncol=2, fontsize=15)

# Remove right and top borders
ax.tick_params(axis='x', width=2, colors='black', labelsize=22, labelcolor='black')
ax.tick_params(axis='y', width=2, colors='black', labelsize=22, labelcolor='black')

# Display figure
plt.tight_layout()
plt.show()

In [2]:
import json

with open('Figure_3_data/PCC.json', 'r') as f:
    loaded_data = json.load(f)

In [3]:
from collections import defaultdict

interval = 50
upper_limit = 700
grouped = defaultdict(list)

for item in loaded_data:
    for k, v in item.items():
        cond_num = float(k)
        if cond_num > upper_limit:
            interval_key = 800
        else:
            interval_key = int(cond_num // interval) * interval
        grouped[interval_key].append(v)
grouped = dict(grouped)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap

# 按 key 排序
sorted_keys = sorted(grouped.keys())

# 构造绘图数据
x_vals = []
y_vals = []
for k in sorted_keys:
    if k < 700:
        label = f"{k}-{k+50}"
    else:
        label = ">700"
    x_vals.extend([label] * len(grouped[k]))  # Interval labels
    y_vals.extend(grouped[k])                  # y coordinates (values)

# Create color gradient
categories = sorted(set(x_vals))
cmap = LinearSegmentedColormap.from_list("custom_cmap", ["#E4CD87", "#C69287"])
colors = [cmap(i / (len(categories) - 1)) for i in range(len(categories))]

# Draw violin plot
plt.figure(figsize=(30,10), dpi=600)
violins = sns.violinplot(x=x_vals, y=y_vals, inner='box', cut=0,inner_kws={
                   'box_width': 3,      # Box plot width
                   'whis_width': 1,     # Whisker width
               })

# Color each violin body (skip extra lines or patches)
# Note: Each violin usually has two patches (body + border), so take every other one
violin_patches = [vp for i, vp in enumerate(violins.collections)]
for i, violin in enumerate(violin_patches):
    violin.set_facecolor(colors[i])
    violin.set_edgecolor('black')
    violin.set_alpha(0.8)

# Add title and labels
plt.xlabel('Condition Number Interval', fontsize=45, labelpad=20)
plt.ylabel('PCC', fontsize=45, labelpad=20)

# Set y-axis range from -1 to 1
plt.ylim(-1, 1)

# Set font size for x and y axis ticks
plt.xticks(fontsize=25, rotation = 45)
plt.yticks(fontsize=25)

plt.show()
